### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


In [13]:
ratings_mat[:,0]

matrix([[ 10.],
        [ 10.],
        [  8.],
        [  9.],
        [ 10.],
        [  6.],
        [  9.],
        [ 10.],
        [  7.],
        [  9.],
        [  9.],
        [  9.],
        [ 10.],
        [  5.],
        [ 10.],
        [  9.],
        [  9.],
        [ 10.],
        [  5.],
        [  8.]])

`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [37]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.sum(np.isnan(ratings_mat)==False) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for i in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        # For each user-movie pair
        for u in range(n_users):
            for m in range(n_movies):                
                # if the rating exists
                actual_rating = ratings_mat[u,m]
                if np.isnan(actual_rating) == False:
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    predict_rating = np.dot(user_mat[u,:], movie_mat[:, m])
                    error = actual_rating - predict_rating
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += error**2
                    # update the values in each matrix in the direction of the gradient
                    user_mat[u,:] = user_mat[u,:] + error*learning_rate*movie_mat[:, m]
                    movie_mat[:, m] = movie_mat[:, m] + error*learning_rate*user_mat[u,:]
        # print results for iteration
        print('Iteration: {}/{}, total SSE: {}'.format(i+1, iters, sse_accum/num_ratings))
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [43]:
user_mat, movie_mat = FunkSVD(ratings_mat, 4, 0.001, 100)# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
Iteration: 1/100, total SSE: 58.35896607478793
Iteration: 2/100, total SSE: 55.715804235841084
Iteration: 3/100, total SSE: 52.94648036321994
Iteration: 4/100, total SSE: 50.05071075133644
Iteration: 5/100, total SSE: 47.03620965319474
Iteration: 6/100, total SSE: 43.918901915069696
Iteration: 7/100, total SSE: 40.72275641969088
Iteration: 8/100, total SSE: 37.479113287768186
Iteration: 9/100, total SSE: 34.22542721825535
Iteration: 10/100, total SSE: 31.003425174553275
Iteration: 11/100, total SSE: 27.85677062006124
Iteration: 12/100, total SSE: 24.828422216274227
Iteration: 13/100, total SSE: 21.957950537283722
Iteration: 14/100, total SSE: 19.279110770819596
Iteration: 15/100, total SSE: 16.817949347654725
Iteration: 16/100, total SSE: 14.591648405944364
Iteration: 17/100, total SSE: 12.608199345281532
Iteration: 18/100, total SSE: 10.866871551554647
Iteration: 19/100, total SSE: 9.359333165583958
Iteration: 20/100, total SSE:

In [44]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.20061059   8.70226649  10.81280319  10.02870806]
 [  8.94722495   7.26648172   8.95687495   8.30942894]
 [  8.32576859   6.72620972   8.96601835   7.69361695]
 [  9.33494221   8.35781411   9.71664526   9.69276455]
 [  8.62617793   7.11891897   9.33186941   8.1973492 ]
 [  6.32738396   5.93248022   7.36959301   6.86280094]
 [  9.21543806   8.11997966   9.38792894   9.3516328 ]
 [  8.3263694    6.90527421   9.08611876   7.90742564]
 [  8.03026394   7.43342144   8.95924274   8.57651174]
 [  8.0357268    6.62454264   8.00521579   7.63825538]
 [  9.44688751   7.61889663   9.16099591   8.76239608]
 [  9.50331895   8.31846393  10.35819546   9.55334806]
 [  9.47262203   8.29424943   9.64261483   9.48040218]
 [  6.72456641   5.86861147   6.32991589   6.74863738]
 [  9.80425378   8.27789449  10.40436634   9.51022115]
 [  9.88204146   8.30150838  10.21497436   9.50353662]
 [  8.59675355   7.03730416   9.12933734   7.99314001]
 [  7.51084615   6.64771112   6.55400913   7.69763673]
 [  7.6602

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [45]:
user_mat, movie_mat = FunkSVD(ratings_mat, 4, 0.005, 250)

Optimization Statistics
Iterations | Mean Squared Error 
Iteration: 1/250, total SSE: 48.90870004494049
Iteration: 2/250, total SSE: 33.62788333489411
Iteration: 3/250, total SSE: 19.0964848344186
Iteration: 4/250, total SSE: 9.379504787994254
Iteration: 5/250, total SSE: 4.80679604243287
Iteration: 6/250, total SSE: 3.1558838861860004
Iteration: 7/250, total SSE: 2.6269363489379605
Iteration: 8/250, total SSE: 2.449205500452513
Iteration: 9/250, total SSE: 2.3767809391253127
Iteration: 10/250, total SSE: 2.3382165709264204
Iteration: 11/250, total SSE: 2.311728479702902
Iteration: 12/250, total SSE: 2.289763240068357
Iteration: 13/250, total SSE: 2.2693162717543762
Iteration: 14/250, total SSE: 2.249062984864262
Iteration: 15/250, total SSE: 2.228377512477656
Iteration: 16/250, total SSE: 2.2069471074635674
Iteration: 17/250, total SSE: 2.1846058219489555
Iteration: 18/250, total SSE: 2.1612593283477883
Iteration: 19/250, total SSE: 2.1368504376981643
Iteration: 20/250, total SSE: 2.1

In [46]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00182786  10.00038492  10.00018694   9.99811812]
 [ 10.00529217   4.0003922    9.00033321   9.99527663]
 [  8.00203517   8.99990621  10.00004364   4.99840952]
 [  9.00042412   8.00063463  10.00011642   9.99905221]
 [ 10.00891752   5.0028372    9.00090664   8.99007528]
 [  6.00602999   4.00162887  10.00053113   5.99353526]
 [  8.99394062   7.99865047   9.99942988   9.00611381]
 [  9.99072786   4.99756824   8.99908224   8.0097567 ]
 [  6.99930036   7.99912385   9.99978213   8.00136375]
 [  8.99021754   4.99749376   8.99907583   7.01028707]
 [  9.01113502   8.00267611  10.00099493   7.9884584 ]
 [  8.990098     9.99830039   9.9992411    9.00961933]
 [  9.99798949   8.99947188   9.99981599   8.00214396]
 [  5.00621639   8.00081816   5.00045354   7.99416651]
 [  9.99866119   7.99905815   9.99977528  10.00194835]
 [  8.99955895   9.00009913   9.99998235  10.00030386]
 [  9.00234331   8.0013497    8.00036073   7.99686503]
 [  9.99952153   7.99986563   0.99993488  10.00052626]
 [  4.9960

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [47]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [48]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, 4, 0.005, 250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
Iteration: 1/250, total SSE: 52.58229892730571
Iteration: 2/250, total SSE: 37.08779408977212
Iteration: 3/250, total SSE: 21.965890560656327
Iteration: 4/250, total SSE: 11.190913476705068
Iteration: 5/250, total SSE: 5.712197870194475
Iteration: 6/250, total SSE: 3.567311366627056
Iteration: 7/250, total SSE: 2.819533357835248
Iteration: 8/250, total SSE: 2.545435221357705
Iteration: 9/250, total SSE: 2.4259354138087814
Iteration: 10/250, total SSE: 2.3613977316891464
Iteration: 11/250, total SSE: 2.318955172631051
Iteration: 12/250, total SSE: 2.28613558493729
Iteration: 13/250, total SSE: 2.257470291106434
Iteration: 14/250, total SSE: 2.2302959389426262
Iteration: 15/250, total SSE: 2.203223973803674
Iteration: 16/250, total SSE: 2.1754940950030592
Iteration: 17/250, total SSE: 2.14667185509416
Iteration: 18/250, total SSE: 2.116498870587203
Iteration: 19/250, total SSE: 2.0848164780502123
Iteration: 20/250, total SSE: 2.051

In [49]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 11.012934510312164:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [58]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, 4, 0.005, 20)#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
Iteration: 1/20, total SSE: 30.58045794304041
Iteration: 2/20, total SSE: 16.164899932027847
Iteration: 3/20, total SSE: 11.456943702387491
Iteration: 4/20, total SSE: 9.066596234846127
Iteration: 5/20, total SSE: 7.570497234324398
Iteration: 6/20, total SSE: 6.52622753704994
Iteration: 7/20, total SSE: 5.745636202022646
Iteration: 8/20, total SSE: 5.133600055935135
Iteration: 9/20, total SSE: 4.636906237338601
Iteration: 10/20, total SSE: 4.223458626819743
Iteration: 11/20, total SSE: 3.87265924289215
Iteration: 12/20, total SSE: 3.570593824625364
Iteration: 13/20, total SSE: 3.3074557620849316
Iteration: 14/20, total SSE: 3.0760832636367823
Iteration: 15/20, total SSE: 2.8710859767756043
Iteration: 16/20, total SSE: 2.6883014486129744
Iteration: 17/20, total SSE: 2.524444780329251
Iteration: 18/20, total SSE: 2.3768757686217885
Iteration: 19/20, total SSE: 2.243439702235702
Iteration: 20/20, total SSE: 2.1223554688218726


In [59]:
preds_1000 = np.dot(user_mat, movie_mat)

`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [60]:
preds_1000

array([[ 4.68724616,  4.04482558,  2.33562537, ...,  3.97799811,
         4.77015202,  3.21158462],
       [ 5.35474771,  4.62324514,  2.66588761, ...,  4.50945327,
         5.48685941,  3.61335097],
       [ 3.80068839,  3.42600953,  2.03862916, ...,  3.24374418,
         3.99965567,  2.34225124],
       ..., 
       [ 3.4254461 ,  2.97774959,  1.66657644, ...,  2.83398684,
         3.56619774,  2.26893204],
       [ 5.74206052,  5.3631051 ,  2.95327171, ...,  5.12762756,
         5.90922742,  3.96469176],
       [ 2.18197395,  1.60112104,  0.9322792 , ...,  1.69757203,
         2.12637028,  1.63460976]])

In [62]:
# Replace each of the comments below with the correct values
num_ratings = np.sum(np.isnan(first_1000_users)==False)# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = np.sum(np.isnan(preds_1000)==False) - np.sum(np.isnan(first_1000_users)==False)# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [63]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
